## Install Dependencies



In [6]:
!pip install -r requirements.txt
import fooddatacentral as fdc
from  scipy.optimize import linprog as lp
import numpy as np
import warnings
import pandas as pd
from eep153_tools.sheets import read_sheets
import plotly.express as px
import cufflinks as cf

## Utilities + Load and Clean Data

In [7]:
# Search for foods in the FDC database
apikey = 'MfcTfizjo11bsqJeJCn9Tb7RdKPQxJRjJSvTKElr' # This is Kaitlin's API key!
food_name = "potato"
fdc.search(apikey,food_name)

,fdcId,description,lowercaseDescription,commonNames,additionalDescriptions,dataType,foodCode,publishedDate,foodCategory,foodCategoryId,allHighlightFields,score,foodNutrients,finalFoodInputFoods,foodMeasures,foodAttributes,foodAttributeTypes,foodVersionIds,ndbNumber,scientificName
0,1103010,Potato patty,potato patty,,potato croquettes;potato cake,Survey (FNDDS),71503010.0,2020-10-30,French fries and other fried white potatoes,998729.0,<b>Includes</b>: <em>potato</em> croquettes;<e...,271.13937,"[{'nutrientId': 1003, 'nutrientName': 'Protein...","[{'foodDescription': 'Egg, whole, raw, fresh',...","[{'disseminationText': '1 cup', 'gramWeight': ...",[],"[{'name': 'Additional Description', 'descripti...",[],NaN,NaN
1,1102089,"Knish, potato","knish, potato",,pastry filled with potato,Survey (FNDDS),58122310.0,2020-10-30,Turnovers and other grain-based items,997808.0,<b>Includes</b>: pastry filled with <em>potato...,268.09576,"[{'nutrientId': 1003, 'nutrientName': 'Protein...","[{'foodDescription': 'Margarine, stick', 'gram...","[{'disseminationText': '1 knish', 'gramWeight'...",[],"[{'name': 'Adjustments', 'description': 'Adjus...",[],NaN,NaN
2,1103052,Potato pancake,potato pancake,,latkes;any source,Survey (FNDDS),71701000.0,2020-10-30,Mashed potatoes and white potato mixtures,998771.0,,259.42587,"[{'nutrientId': 1003, 'nutrientName': 'Protein...","[{'foodDescription': 'Salt, table, iodized', '...","[{'disseminationText': '1 surface inch', 'gram...",[],"[{'name': 'Adjustments', 'description': 'Adjus...",[],NaN,NaN
3,1103054,Stewed potatoes,stewed potatoes,,papas guisadas;with or without fat added;with ...,Survey (FNDDS),71703990.0,2020-10-30,Mashed potatoes and white potato mixtures,998773.0,,259.42587,"[{'nutrientId': 1003, 'nutrientName': 'Protein...","[{'foodDescription': 'Vegetable oil, NFS', 'gr...","[{'disseminationText': '1 cup', 'gramWeight': ...",[],"[{'name': 'Additional Description', 'descripti...",[],NaN,NaN
4,1102879,"Potato, NFS","potato, nfs",,,Survey (FNDDS),71000100.0,2020-10-30,"White potatoes, baked or boiled",998598.0,,257.77097,"[{'nutrientId': 1003, 'nutrientName': 'Protein...","[{'foodDescription': 'Potato, boiled, from fre...","[{'disseminationText': '1 cup', 'gramWeight': ...",[],"[{'name': 'Attribute', 'description': 'Generic...",[],NaN,NaN
5,167943,"Bread, potato","bread, potato",,,SR Legacy,NaN,2019-04-01,Baked Products,NaN,,257.69394,"[{'nutrientId': 1057, 'nutrientName': 'Caffein...",[],[],[],[],[],18971.0,NaN
6,1100670,"Bread, potato","bread, potato",,pre-sliced or not,Survey (FNDDS),51127010.0,2020-10-30,Yeast breads,996389.0,,257.69394,"[{'nutrientId': 1003, 'nutrientName': 'Protein...","[{'foodDescription': 'Bread, potato', 'gramWei...",[{'disseminationText': '1 large or thick slice...,[],"[{'name': 'Additional Description', 'descripti...",[],NaN,NaN
7,1102087,"Gnocchi, potato","gnocchi, potato",,,Survey (FNDDS),58122220.0,2020-10-30,"Pasta mixed dishes, excludes macaroni and cheese",997806.0,,257.69394,"[{'nutrientId': 1003, 'nutrientName': 'Protein...","[{'foodDescription': 'Butter, salted', 'gramWe...","[{'disseminationText': '1 cup', 'gramWeight': ...",[],"[{'name': 'Adjustments', 'description': 'Adjus...",[],NaN,NaN
8,1103060,Potato chowder,potato chowder,,corn chowder,Survey (FNDDS),71803010.0,2020-10-30,Soups,998779.0,,257.69394,"[{'nutrientId': 1003, 'nutrientName': 'Protein...","[{'foodDescription': 'Onions, raw', 'gramWeigh...","[{'disseminationText': '1 cup', 'gramWeight': ...",[],"[{'name': 'Adjustments', 'description': 'Adjus...",[],NaN,NaN
9,168446,Potato flour,potato flour,,,SR Legacy,NaN,2019-04-01,Vegetables and Vegetable Products,NaN,,257.69394,"[{'nutrientId': 1186, 'nutrientName': 'Folic a...",[],[],[],[],[],11413.0,NaN


In [8]:
# Extract food's nutrition info using the FDC ID
id =  1102879
fdc.nutrients(apikey,fdc_id=id)

,Quantity,Units
Proximates,0.00,g
Water,72.09,g
Energy,125.00,kcal
Protein,1.87,g
Total lipid (fat),4.24,g
...,...,...
Cholesterol,3.00,mg
Amino acids,0.00,g
"Alcohol, ethyl",0.00,g
Caffeine,0.00,mg


In [9]:
# Read google spreadsheet with food/price info
SHEETs = [("https://docs.google.com/spreadsheets/d/1j38SlusoWvBtohgU-PAondBKihr1tCpjfLcgit7Ic_s/edit#gid=0",
           "Berkeley, Whole"), 
         ]
df = read_sheets(SHEETs[0][0])[SHEETs[0][1]]
df = df.drop(['Name of person', 'Link'], axis=1)

Key available for students@eep153.iam.gserviceaccount.com.


In [10]:
# Create a dataframe with selected food nutrition info
D = {}
count = 0
for food in df.Food.tolist():
    try:
        FDC = df.loc[df.Food==food,:].FDC[count]
        count+=1
        D[food] = fdc.nutrients(apikey,FDC).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))        

FoodNutrients = pd.DataFrame(D,dtype=float)

In [11]:
# Convert food quantities to FDC units
df['FDC Quantity'] = df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

# Now may want to filter df by time or place--need to get a unique set of food names.
df['FDC Price'] = df['Price']/df['FDC Quantity']

df.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
Prices = df.groupby('Food',sort=False)['FDC Price'].min()

/opt/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1990: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



# Deliverable: Dietary Reference Intakes

In [12]:
def DRI(age, sex):
    '''
    Description: Function that retrieves Dietary Reference Intakes or Recommended Daily Allowances for a
        population of interest described by the input characteristics

    Inputs:
        Age: The age of interest, in years (integer)
        Sex: The sex of interest, 'M' or 'F' (string)
        
    Output: Returns a pandas.Series of DRI or RDA of a variety of nutrients for our population of interest
    '''
    
    # Make a combined dataframe with min and max nutrients
    nut_min = pd.read_csv('./diet_minimums.csv').set_index('Nutrition').iloc[:, 2:]
    nut_max = pd.read_csv('./diet_maximums.csv').set_index('Nutrition').iloc[:, 2:]
    nut = pd.concat([nut_min, -nut_max])
    
    # Case 1: Child
    if age in range(1, 4):
        return nut.loc[:,'C 1-3']
    
    # Case 2: Not a child
    # Determine the age-range; Filter data by desired age-range and sex
    age_ranges = [[1,4], [4,9], [9,14], [14,19], [19,31], [31,51], [51,1000]]
    for r in age_ranges:
        if age in range(r[0], r[1]):
            return nut.filter(regex = sex).filter(regex = ' ' + str(r[0])).iloc[:, 0]
        
def grouping(age, sex):
    # Case 1: Child
    if age in range(1, 4):
        return 'C 1-3'
    
    # Case 2: Not a child
    # Determine the age-range; Filter data by desired age-range and sex
    age_ranges = [[1,4], [4,9], [9,14], [14,19], [19,31], [31,51], [51,1000]]
    for r in age_ranges:
        if age in range(r[0], r[1]):
            return sex + ' ' + str(r[0]) + '-' + str(r[1]-1)

# Deliverable: Subsistence Solver

In [13]:
def solve_subsistence_problem(FoodNutrients,Prices,diet_min,diet_max,max_weight=23,tol=1e-6):
    """Solve Stigler's Subsistence Cost Problem.

    Inputs:
       - FoodNutrients : A pd.DataFrame with rows corresponding to foods, columns to nutrients.
       - Prices : A pd.Series of prices for different foods
       - diet_min : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing minimum intakes.
       - diet_max : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing maximum intakes.
       - max_weight : Maximum weight (in hectograms) allowed for diet.
       - tol : Solution values smaller than this in absolute value treated as zeros.
       
    """
    p = Prices.apply(lambda x:x.magnitude).dropna()

    # Compile list that we have both prices and nutritional info for; drop if either missing
    use = p.index.intersection(FoodNutrients.columns)
    p = p[use]

    # Drop nutritional information for foods we don't know the price of,
    # and replace missing nutrients with zeros.
    Aall = FoodNutrients[p.index].fillna(0)

    # Drop rows of A that we don't have constraints for.
    Amin = Aall.loc[Aall.index.intersection(diet_min.index)]

    Amax = Aall.loc[Aall.index.intersection(diet_max.index)]

    # Minimum requirements involve multiplying constraint by -1 to make <=.
    A = pd.concat([Amin,
                   -Amax])

    b = pd.concat([diet_min,
                   -diet_max]) # Note sign change for max constraints

    # Make sure order of p, A, b are consistent
    A = A.reindex(p.index,axis=1)
    A = A.reindex(b.index,axis=0)

    if max_weight is not None:
        # Add up weights of foods consumed
        A.loc['Hectograms'] = -1
        b.loc['Hectograms'] = -max_weight
        
    # Now solve problem!  (Note that the linear program solver we'll use assumes
    # "less-than-or-equal" constraints.  We can switch back and forth by
    # multiplying $A$ and $b$ by $-1$.)

    result = lp(p, -A, -b, method='interior-point')

    result.A = A
    result.b = b
    
    if result.success:
        result.diet = pd.Series(result.x,index=p.index)
    else: # No feasible solution?
        warnings.warn(result.message)
        result.diet = pd.Series(result.x,index=p.index)*np.nan  

    return result

## Dietary Requirements



In [14]:
# Read Dietary Requirements sheet from class
DRI_url = "https://docs.google.com/spreadsheets/d/1XJRHTnxNJwrUXperIhwrwDp1HcVxPEVoQobYDmjg9Qw/"

DRIs = read_sheets(DRI_url)

# Define *minimums*
diet_min = DRIs['diet_minimums'].set_index('Nutrition')

# Define *maximums*
diet_max = DRIs['diet_maximums'].set_index('Nutrition')

# diet max but with forgiving values - basically forces the output to tell us the end values for sugar, chol, salt
diet_max_forgiving = DRIs['diet_maximums_forgiving'].set_index('Nutrition')

Key available for students@eep153.iam.gserviceaccount.com.


## Solving for the Minimum Cost Diet


In [15]:
# Select a group of people to investigate
group = 'M 19-30'
tol = 1e-6

result = solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol)

print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())

Cost of diet for M 19-30 is $3.30 per day.


Diet (in 100s of grams or milliliters):
egg              0.183548
broccoli         0.587432
carrots          0.790452
peanut butter    0.707015
whole milk       3.763467
almond milk      0.534842
potatoes         1.254121
black beans      4.302618
oats             2.248025
spinach          0.050269
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          2400.000007          2400.0
Protein                          103.431080            56.0
Fiber, total dietary              75.162112            33.6
Folate, DFE                      571.528723           400.0
Calcium, Ca                     1000.000007          1000.0
Carbohydrate, by difference      327.917661           130.0
Iron, Fe                          24.471712             8.0
Magnesium, Mg                    755.921109  

## Effects of Price Changes on Subsistence Diet Cost



As prices change, we should expect the minimum cost diet to also
  change.  The code below creates a graph which changes prices away
  from the \`base&rsquo; case one food at a time, and plots changes in total
  diet cost.



In [16]:
cf.go_offline()

scale = [.5,.6,.7,.8,.9,1.,1.1,1.2,1.3,1.4,1.5]

cost0 = solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol).fun

# Find the diet cost response to price
Price_response={}
for s in scale:
    cost = {}
    for i,p in enumerate(Prices):
        my_p = Prices.copy()
        my_p[i] = p*s
        result = solve_subsistence_problem(FoodNutrients,my_p,diet_min[group],diet_max[group],tol=tol)
        cost[Prices.index[i]] = np.log(result.fun/cost0)
    Price_response[np.log(s)] = cost


# Get list of foods in the min cost diet, just plot these foods
Diet_foods = list(diet[diet >= tol].index)

# Make the line plot
Price_response = pd.DataFrame(Price_response).T
Price_response[Diet_foods].iplot(xTitle='Change in log Price',yTitle='Change in log Cost')

## Effects of Price Changes on Subsistence Diet Composition



The code below creates a graph which changes prices just for *one* food,
  and traces out the effects of this change on all the foods consumed.



In [17]:
cf.go_offline()

# Specify the reference good
ReferenceGood = 'almond milk'

scale = [0.5,0.75,0.9,1.,1.1,1.2,1.3,1.4,1.5,2,4]

cost0 = solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol).fun

my_p = Prices.copy()

# Find the diet composition response to price
diet = {}
for s in scale:

    my_p[ReferenceGood] = Prices[ReferenceGood]*s
    result = solve_subsistence_problem(FoodNutrients,my_p,diet_min[group],diet_max[group],tol=tol)
    diet[my_p[ReferenceGood]] = result.diet

Diet_response = pd.DataFrame(diet).T
Diet_response.index.name = '%s Price' % ReferenceGood

Diet_response.reset_index(inplace=True)

# Get rid of units for index (cufflinks chokes)
Diet_response['%s Price' % ReferenceGood] = Diet_response['%s Price' % ReferenceGood].apply(lambda x: x.magnitude)

Diet_response = Diet_response.set_index('%s Price' % ReferenceGood)

# Just look at goods consumed in quantities greater than error tolerance
Diet_response.loc[:,(Diet_response>tol).sum()>0][Diet_foods].iplot(xTitle='%s Price' % ReferenceGood.title(),yTitle='Hectograms')

/opt/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1990: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



## Effects of Price Changes on Subsistence Diet Nutrition



The code below creates a graph which uses the food price changes
  described above, but maps into nutrients.



In [18]:
# Matrix product maps quantities of food into quantities of nutrients
NutrientResponse = Diet_response@FoodNutrients.T

# Drop columns of missing nutrients, plot first half of nutrients (avoid display issues)
NutrientResponse = NutrientResponse.loc[:,NutrientResponse.count()>0]
NutrientResponse.iloc[:,0:17].iplot(xTitle='%s Price' % ReferenceGood.title(),yTitle='Hectograms')

In [19]:
# Matrix product maps quantities of food into quantities of nutrients
NutrientResponse = Diet_response@FoodNutrients.T

# Drop columns of missing nutrients, plot second half of nutrients (avoid display issues)
NutrientResponse = NutrientResponse.loc[:,NutrientResponse.count()>0]
NutrientResponse.iloc[:,17:34].iplot(xTitle='%s Price' % ReferenceGood.title(),yTitle='Hectograms')

## Investigate Nutrient Profile of Specific Foods ##

In [20]:
# Specify foods of interest
ref_foods = ['potatoes','almond milk','black beans','oats','whole milk']

# Eliminate rows with all zeroes | any Nans
ref_nutrients = FoodNutrients[ref_foods]
ref_nutrients = ref_nutrients.loc[(ref_nutrients != 0).all(axis=1), :]
ref_nutrients = ref_nutrients.loc[np.isnan(ref_nutrients).any(axis=1) == False, :]

# Normalize each row (nutrient amounts sum to 1)
ref_nutrients = ref_nutrients.transpose()
column_sums = ref_nutrients.sum()
norm_nutrients = ref_nutrients/column_sums
norm_nutrients = norm_nutrients.transpose()
norm_nutrients

,potatoes,almond milk,black beans,oats,whole milk
"Calcium, Ca",0.012385,0.427793,0.126331,0.128809,0.304682
"Carbohydrate, by difference",0.174047,0.002869,0.207154,0.576184,0.039746
"Copper, Cu",0.226853,0.023642,0.313591,0.434802,0.001112
"Fatty acids, total monounsaturated",0.307394,0.147001,0.007463,0.399371,0.138771
"Fatty acids, total polyunsaturated",0.313031,0.066175,0.044069,0.550859,0.025866
"Fatty acids, total saturated",0.274066,0.023667,0.025081,0.253090,0.424096
"Magnesium, Mg",0.103544,0.029273,0.220031,0.595379,0.051772
Niacin,0.378894,0.021286,0.247827,0.321946,0.030048
"Phosphorus, P",0.064184,0.039914,0.212609,0.548238,0.135054
"Potassium, K",0.271972,0.022511,0.331191,0.264661,0.109666


In [21]:
# Customize colors to match the previous line graphs
C = [0]*5
C[0] = 'gold'
C[1] = 'teal'
C[2] = 'olive'
C[3] = 'coral'
C[4] = 'crimson'

# Make the bar graph
px.bar(norm_nutrients, labels={'index':'Nutrients', 'value':'Normalized Amount'}, color_discrete_sequence=C)

## Adding Constraint on Total Weight



At least at some prices the minimum cost subistence diet involves
eating unreasonable amounts of food (e.g., 10 kilograms of cabbage per
day).  We can easily add an additional constraint of the form
$$
     \sum x_i \leq \text{max weight}
$$
to our linear programming problem since it&rsquo;s just another linear
inequality, and this may give us more realistic results.



### Price Changes and Subsistence Diet Composition with Weight Constraint



Re-do our analysis of changing prices, but with a constraint that
  total weight of diet must be less that 12 hectograms (1.2 kg).



In [22]:
cf.go_offline()

# Specify reference good
ReferenceGood = 'almond milk'

scale = [0.5,0.75,0.9,1.,1.1,1.2,1.3,1.4,1.5,2,4]

cost0 = solve_subsistence_problem(FoodNutrients,Prices,
                                  diet_min[group],diet_max[group],max_weight=10,tol=tol).fun

my_p = Prices.copy()

diet = {}
for s in scale:

    my_p[ReferenceGood] = Prices[ReferenceGood]*s
    result = solve_subsistence_problem(FoodNutrients,my_p,
                                       diet_min[group],diet_max[group],max_weight=10,tol=tol)
    diet[my_p[ReferenceGood]] = result.diet

Diet_response = pd.DataFrame(diet).T
Diet_response.index.name = '%s Price' % ReferenceGood

Diet_response.reset_index(inplace=True)

# Get rid of units for index (cufflinks chokes)
Diet_response['%s Price' % ReferenceGood] = Diet_response['%s Price' % ReferenceGood].apply(lambda x: x.magnitude)

Diet_response = Diet_response.set_index('%s Price' % ReferenceGood)

# Just look at goods consumed in quantities greater than error tolerance
Diet_response.loc[:,(Diet_response>tol).sum()>0].iplot(xTitle='%s Price' % ReferenceGood,yTitle='Hectograms')

/opt/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1990: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.

